In [1]:
# Import essential packages
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
from bs4 import BeautifulSoup
import pandas

In [2]:
# Get html source from certain web page
def load_page(page_num:int):
    driver.get(url=base_url+"?page="+str(base_page))
    sleep(2.5)
    soup_obj=BeautifulSoup(driver.page_source,'lxml')
    return soup_obj

In [3]:
# Detect if there's a next page
def go_next(soup_obj:BeautifulSoup):
    next_page_disabled = soup_obj.find(name='a',class_='glyphicon glyphicon-menu-right').attrs.get('disabled')
    if next_page_disabled=='false':
        return True
    else:
        return False

In [4]:
def process_with_pandas(soup_obj:BeautifulSoup,store:list):
    items = soup_obj.find_all(name='div',attrs={'class':'caption'})

    for item in items:
        if item.find(class_='product-rating') is not None:
            rating=float(item.find(class_='product-rating').get_text())
        else: rating=0.0 # refers to no rating
        info_dict = {'name':item.h3.get_text(),'html_link':item.h3.a.attrs['href'],'price':item.find(class_="price").get_text(),'rating':rating}
        store.append(info_dict)


In [5]:
# Initial Setup
base_url="https://hobbii.fr/fils"
base_page=1
driver = webdriver.Chrome(executable_path="Users/luciezhang/Téléchargements/chromedriver_mac64/chromedriver.exe")
storage_pd = list()

/var/folders/gd/rmmjkh353fvb_1jfzg6qk6th0000gn/T/ipykernel_24947/251371151.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="Users/luciezhang/Téléchargements/chromedriver_mac64/chromedriver.exe")


In [6]:
# Loop through available pages and store results
while True:
    soup=load_page(base_page)
    process_with_pandas(soup,storage_pd)
    print("page number "+str(base_page)+" has finished\n")
    base_page=base_page+1

    if not go_next(soup):
        break

page number 1 has finished

page number 2 has finished

page number 3 has finished

page number 4 has finished



In [7]:
df = pandas.DataFrame(storage_pd)

In [8]:
df

,name,html_link,price,rating
0,Friends Cotton 8/4 - Friends,https://hobbii.fr/friends-cotton-8-4,"À partir de 1,40 €",4.9
1,Rainbow Cotton 8/4 - Hobbii,https://hobbii.fr/rainbow-cotton-8-4,"À partir de 1,66 €",5.0
2,Amigo - Hobbii,https://hobbii.fr/amigo,"1,75 € 3,65 €",4.9
3,Baby Snuggle Solid - Hobbii,https://hobbii.fr/baby-snuggle-solid,"À partir de 5,20 €",4.8
4,Friends Cotton 8/8 - Friends,https://hobbii.fr/friends-cotton-8-8,"À partir de 1,40 €",4.6
...,...,...,...,...
234,Cotton 8/4 Soft Print Color Pack 02 - Cotton K...,https://hobbii.fr/cotton-8-4-soft-print-wonder...,"À partir de 16,90 €",0.0
235,Cotton 8/4 Print Color Pack (Couleurs 11-20) -...,https://hobbii.fr/wonderbag-cotton-8-4-print-c...,"À partir de 18,85 €",5.0
236,Friends Kid Silk - Dark Gray - Article défectu...,https://hobbii.fr/friends-kid-silk-dark-gray-a...,33 € 34 €,5.0
237,Reborn Denim Uni - Kremke Soul Wool,https://hobbii.fr/reborn-denim-uni,"4,90 €",0.0


In [9]:
# process the price string to float value of price
# for those having a discount, calculate discount percentage
for i in range(0,len(df)):
    s = df['price'][i]
    if s[2]=='p':
        df['price'][i] = float(s[12:-2].replace(',','.'))
        df.loc[i,'discounted'] = 0.0
    else:
        df['price'][i] = float(s.split(' €')[0].replace(',','.'))
        if len(s.split(' €'))>2:
            df.loc[i,'discounted'] = 1-float(s.split(' €')[0].replace(',','.'))/float(s.split(' €')[1].replace(',','.'))
        else: df.loc[i,'discounted'] = 0.0

df=df.round({'discounted':2})

/var/folders/gd/rmmjkh353fvb_1jfzg6qk6th0000gn/T/ipykernel_24947/1377342718.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price'][i] = float(s[12:-2].replace(',','.'))
/var/folders/gd/rmmjkh353fvb_1jfzg6qk6th0000gn/T/ipykernel_24947/1377342718.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price'][i] = float(s[12:-2].replace(',','.'))
/var/folders/gd/rmmjkh353fvb_1jfzg6qk6th0000gn/T/ipykernel_24947/1377342718.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in

In [11]:
#find product with good rating
user_rating=input("enter a min rating:")
print("showing product with better rating than: "+user_rating)
df[df['rating']>=float(user_rating)]

enter a min rating:4.5
showing product with better rating than: 4.5


,name,html_link,price,rating,discounted
0,Friends Cotton 8/4 - Friends,https://hobbii.fr/friends-cotton-8-4,1.4,4.9,0.00
1,Rainbow Cotton 8/4 - Hobbii,https://hobbii.fr/rainbow-cotton-8-4,1.66,5.0,0.00
2,Amigo - Hobbii,https://hobbii.fr/amigo,1.75,4.9,0.52
3,Baby Snuggle Solid - Hobbii,https://hobbii.fr/baby-snuggle-solid,5.2,4.8,0.00
4,Friends Cotton 8/8 - Friends,https://hobbii.fr/friends-cotton-8-8,1.4,4.6,0.00
...,...,...,...,...,...
232,Bead Deluxe - Go Handmade,https://hobbii.fr/bead-deluxe,3.36,4.5,0.00
233,Metallic Thread - Madeira,https://hobbii.fr/madeira-metallic,3.3,5.0,0.00
235,Cotton 8/4 Print Color Pack (Couleurs 11-20) -...,https://hobbii.fr/wonderbag-cotton-8-4-print-c...,18.85,5.0,0.00
236,Friends Kid Silk - Dark Gray - Article défectu...,https://hobbii.fr/friends-kid-silk-dark-gray-a...,33.0,5.0,0.03


In [12]:
# find a product with good discount
user_discount=input("enter a min discount:")
print("showing product with larger discount than: "+user_discount)
df[df['discounted']>=float(user_discount)]

enter a min discount:0.45
showing product with larger discount than: 0.45


,name,html_link,price,rating,discounted
2,Amigo - Hobbii,https://hobbii.fr/amigo,1.75,4.9,0.52
14,Ribbon - Hobbii,https://hobbii.fr/ribbon-hobbii,7.0,4.9,0.50
20,Horizon - Hobbii,https://hobbii.fr/horizon,7.9,4.9,0.60
36,Black Friday Cotton 8/4 Color Pack (Limited Ed...,https://hobbii.fr/black-friday-2022-cotton-8-4,12.2,4.8,0.45
39,Universe XL - Hobbii,https://hobbii.fr/universe-xl,5.6,4.9,0.53
45,Starlight Soft - Hobbii,https://hobbii.fr/starlight-soft,2.45,4.4,0.67
64,Starlight - Hobbii,https://hobbii.fr/hobbii-starlight,3.45,4.1,0.57
135,Black Friday Sock Yarn Color Pack (Édition lim...,https://hobbii.fr/black-friday-sock-yarn-color...,21.0,0.0,0.51
138,Azalea - Hobbii,https://hobbii.fr/azalea,11.1,4.7,0.47
142,Universe Halloween (Édition limitée) - Hobbii,https://hobbii.fr/universe-halloween,6.0,5.0,0.50


In [13]:
#find product with low price
user_price= input("enter a max price:")
print("showing product with lower price than "+user_price)
df[df['price']<=float(user_price)]

enter a max price:10
showing product with lower price than 10


,name,html_link,price,rating,discounted
0,Friends Cotton 8/4 - Friends,https://hobbii.fr/friends-cotton-8-4,1.4,4.9,0.00
1,Rainbow Cotton 8/4 - Hobbii,https://hobbii.fr/rainbow-cotton-8-4,1.66,5.0,0.00
2,Amigo - Hobbii,https://hobbii.fr/amigo,1.75,4.9,0.52
3,Baby Snuggle Solid - Hobbii,https://hobbii.fr/baby-snuggle-solid,5.2,4.8,0.00
4,Friends Cotton 8/8 - Friends,https://hobbii.fr/friends-cotton-8-8,1.4,4.6,0.00
...,...,...,...,...,...
226,"Macramé Deluxe, 5mm - Go Handmade",https://hobbii.fr/macrame-deluxe-5mm,9.41,0.0,0.00
231,Bloom - Cewec,https://hobbii.fr/bloom,9.75,0.0,0.00
232,Bead Deluxe - Go Handmade,https://hobbii.fr/bead-deluxe,3.36,4.5,0.00
233,Metallic Thread - Madeira,https://hobbii.fr/madeira-metallic,3.3,5.0,0.00
